In [35]:
#from google.colab import auth
#auth.authenticate_user()

In [36]:
!pip install imblearn

In [37]:
from google.cloud import bigquery
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import f_classif, chi2, SelectKBest
from sklearn.decomposition import PCA
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier



In [38]:
def create_training_set (df, df_webtable,df_loyaltytable, FirstFold) :
  df=df_mastertable[df_mastertable['customer_id'].isin(df_webtable[df_webtable['date_of_visit'].between(FirstFold['training_start_date'],FirstFold['training_end_date'])]['customer_id'])]
  df=df[~df['customer_id'].isin(df_loyaltytable[df_loyaltytable['enrollment_date'].between(FirstFold['training_start_date'],FirstFold['training_end_date'])]['customer_id'])]
  return df

In [39]:
def create_training_labels (df,df_loyaltytable, fold):
  enrolled =df_loyaltytable[df_loyaltytable['customer_id'].isin(df_loyaltytable[df_loyaltytable['enrollment_date'].between(fold['label_start_date'],fold['label_end_date'])]['customer_id'])]['customer_id']
  df['target'] = df['customer_id'].isin(enrolled)
  return df['target']

In [40]:
def create_test_set (df, df_webtable,df_loyaltytable, fold) :
  test_df = df_mastertable[df_webtable['date_of_visit'].between(fold['test_start_date'],fold['test_end_date'])]
  customer_ids_enrolled_already = df_loyaltytable[df_loyaltytable['enrollment_date']<=fold['test_end_date']]
  test_df = test_df[~test_df['customer_id'].isin(customer_ids_enrolled_already['customer_id'])]
  return test_df

In [41]:
def create_test_labels(test_df,fold,df_loyaltytable) :
  # 'test_start_date' : pd.to_datetime('2025-04-01'), 'test_end_date' : pd.to_datetime('2025-04-15'),'test_label_start_date' : , 'test_label_end_date' : pd.to_datetime('2025-04-30')}
  enrolled =df_loyaltytable[df_loyaltytable['customer_id'].isin(df_loyaltytable[df_loyaltytable['enrollment_date'].between(pd.to_datetime('2025-04-16'),pd.to_datetime('2025-04-30'))]['customer_id'])]
  test_df['target'] = test_df['customer_id'].isin(enrolled['customer_id'])
  return test_df['target']

In [42]:
def keep_relevant_cols(df) :
  rem_cols =['customer_id','target']
  cols = [i for i in df.columns if i not in rem_cols]
  return df[cols], df[rem_cols]

In [43]:
def standardization (df) :
  scaler = StandardScaler()
  scaler.fit(df.select_dtypes(include='number'))
  df[df.select_dtypes(include='number').columns] = scaler.transform(df[df.select_dtypes(include='number').columns])
  return df

In [44]:
def Onehotencoding (df):
  OE = OneHotEncoder(sparse_output=False,drop='first')
  OE.fit(df[df.select_dtypes(include='object').columns])
  df[OE.get_feature_names_out()] = OE.transform(df[df.select_dtypes(include='object').columns])
  df.drop(columns=df.select_dtypes(include='object').columns,inplace=True)
  return df, OE

In [45]:
def selecting_best_features(df,target,ObjOE) :
   num_feature_length = len(df_mastertable.select_dtypes(include = 'number').columns)
   selector_num = SelectKBest(f_classif, k=3)
   num_features = selector_num.fit(df,target['target']).get_feature_names_out()
   cat_feature_length = len(df[ObjOE.get_feature_names_out()])
   selector_cat = SelectKBest(chi2, k=6)
   cat_features = selector_cat.fit(df[list(ObjOE.get_feature_names_out()) + ['email_provided']],target['target']).get_feature_names_out()
   features = list(num_features) + list(cat_features)
   return df[features], target


In [46]:
def folds_properties(first_date, last_date, step=10,
                     training_size=30, training_label_size=30,
                     test_size=15, test_label_size=15):

    max_days_needed = training_size + training_label_size + test_size + test_label_size
    total_days = (last_date - first_date).days
    num_of_folds = (total_days - max_days_needed) // step + 1

    dict_folds = {}

    for i in range(num_of_folds):
        training_start_date = first_date + pd.Timedelta(days=i * step)
        training_end_date = training_start_date + pd.Timedelta(days=training_size - 1)

        label_start_date = training_end_date + pd.Timedelta(days=1)
        label_end_date = label_start_date + pd.Timedelta(days=training_label_size - 1)

        test_start_date = label_end_date + pd.Timedelta(days=1)
        test_end_date = test_start_date + pd.Timedelta(days=test_size - 1)

        test_label_start_date = test_end_date + pd.Timedelta(days=1)
        test_label_end_date = test_label_start_date + pd.Timedelta(days=test_label_size - 1)

        if test_label_end_date > last_date:
            break

        k = f'Fold {i + 1}'
        dict_folds[k] = {
            'training_start_date': training_start_date,
            'training_end_date': training_end_date,
            'label_start_date': label_start_date,
            'label_end_date': label_end_date,
            'test_start_date': test_start_date,
            'test_end_date': test_end_date,
            'test_label_start_date': test_label_start_date,
            'test_label_end_date': test_label_end_date
        }

    return dict_folds, num_of_folds


In [47]:
def  resample(X_set,Y_set) :
  rus = RandomUnderSampler(sampling_strategy=0.6)
  X_resampled, y_resampled = rus.fit_resample(X_set, Y_set)
  return X_resampled, y_resampled

In [48]:
bq_client = bigquery.Client(project = 'customer-marketing-analytics')
querymastertable = 'Select * from customer-marketing-analytics.customer_and_marketing_data_analysis.MASTER_CUSTOMER_TABLE'
querywebtable = 'Select customer_id,date_of_visit from customer-marketing-analytics.customer_and_marketing_data_analysis.Web_analytics_Data_CMA'
queryloyaltytable = 'Select customer_id, enrollment_status, enrollment_date from customer-marketing-analytics.customer_and_marketing_data_analysis.Loyalty_Program_Data_CMA'


resmaster = bq_client.query(querymastertable)
resweb = bq_client.query(querywebtable)
resloyalty = bq_client.query(queryloyaltytable)

In [49]:
df_mastertable = resmaster.to_dataframe()
df_webtable = resweb.to_dataframe()
df_loyaltytable = resloyalty.to_dataframe()

In [50]:
first_date = df_webtable['date_of_visit'].min( )
last_date = df_webtable['date_of_visit'].max( )

In [51]:
fold_dictionary,num_folds = folds_properties(first_date=first_date,last_date=last_date)

In [52]:
def dataset_creator(fold_dictionary,num_folds) :
  dict_tables = {}
  for i in range(num_folds):
    k = f'fold_datasets_{i+1}'
    training_df = create_training_set(df_mastertable,df_webtable,df_loyaltytable,fold_dictionary[f'Fold {i+1}'])
    training_labels = create_training_labels(training_df,df_loyaltytable,fold_dictionary[f'Fold {i+1}'])
    test_df = create_test_set(df_mastertable,df_webtable,df_loyaltytable,fold_dictionary[f'Fold {i+1}'])
    test_labels = create_test_labels(test_df,fold_dictionary[f'Fold {i+1}'],df_loyaltytable)
    training_df, training_labels = keep_relevant_cols(training_df)
    test_df, test_labels = keep_relevant_cols(test_df)
    training_df = standardization(training_df)
    test_df = standardization(test_df)
    training_df,OE = Onehotencoding(training_df)
    test_df,OET = Onehotencoding(test_df)
    training_df, training_labels = selecting_best_features(training_df,training_labels,OE)
    test_df=test_df[training_df.columns]
    dict_tables[k] = {'training_df':training_df,'training_labels':training_labels,'test_df':test_df,'test_labels':test_labels}
  return dict_tables






In [53]:
datasets = dataset_creator(fold_dictionary,num_folds)

/var/tmp/ipykernel_2342/3307221883.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test_df = df_mastertable[df_webtable['date_of_visit'].between(fold['test_start_date'],fold['test_end_date'])]
/var/tmp/ipykernel_2342/3307221883.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test_df = df_mastertable[df_webtable['date_of_visit'].between(fold['test_start_date'],fold['test_end_date'])]
/var/tmp/ipykernel_2342/3307221883.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test_df = df_mastertable[df_webtable['date_of_visit'].between(fold['test_start_date'],fold['test_end_date'])]
/var/tmp/ipykernel_2342/3307221883.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test_df = df_mastertable[df_webtable['date_of_visit'].between(fold['test_start_date'],fold['test_end_date'])]


In [54]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix
gbc = GradientBoostingClassifier(n_estimators=1000, loss ='exponential',learning_rate=0.05,subsample=0.75,min_samples_leaf=2,min_samples_split=4,random_state=42)

In [55]:
accuracy_metrics=list( )
timestamp = pd.Timestamp.now( )
for j in range(0,num_folds) :
  X,Y = resample(datasets[f'fold_datasets_{j+1}']['training_df'],datasets[f'fold_datasets_{j+1}']['training_labels']['target'])
  gbc.fit(X,Y)
  gbc_pred = gbc.predict(datasets[f'fold_datasets_{j+1}']['test_df'])
  mat=confusion_matrix(datasets[f'fold_datasets_{j+1}']['test_labels']['target'],gbc_pred)
  accuracy_metrics.append(list(mat.flatten( ))+[timestamp])
  print(mat)

[[432 124]
 [  5  25]]
[[487 105]
 [ 12  18]]
[[623 131]
 [  9  22]]
[[1051  254]
 [  10   47]]


In [56]:
accuracy_tracking = pd.DataFrame(data = accuracy_metrics, columns =['True negatives', 'False negatives', 'False Positives','True Positives','Timestamp'])
print(accuracy_tracking)

   True negatives  False negatives  False Positives  True Positives  \
0             432              124                5              25   
1             487              105               12              18   
2             623              131                9              22   
3            1051              254               10              47   

                   Timestamp  
0 2025-04-12 10:43:37.425753  
1 2025-04-12 10:43:37.425753  
2 2025-04-12 10:43:37.425753  
3 2025-04-12 10:43:37.425753  


In [57]:
final_model = GradientBoostingClassifier(n_estimators=1000, loss ='exponential',learning_rate=0.05,subsample=0.75,min_samples_leaf=2,min_samples_split=4)

In [58]:
prod_training_fold ={'training_start_date' :last_date-pd.Timedelta(days=45), 'training_end_date' : last_date-pd.Timedelta(days=15),'label_start_date' : last_date-pd.Timedelta(days=14),'label_end_date': last_date}

In [59]:
prod_training_df = create_training_set(df_mastertable,df_webtable,df_loyaltytable,prod_training_fold)
prod_training_df['target'] = create_training_labels(prod_training_df,df_loyaltytable,prod_training_fold)
prod_training_df, prod_training_labels = keep_relevant_cols(prod_training_df)
prod_training_df = standardization(prod_training_df)
prod_training_df,OE = Onehotencoding(prod_training_df)
prod_training_df, prod_training_labels = selecting_best_features(prod_training_df,prod_training_labels,OE)


In [60]:
pred_against_cids = pd.DataFrame()
prod_pred_fold ={'test_start_date' : last_date-pd.Timedelta(days=14),'test_end_date': last_date}
prod_pred_df = create_test_set(df_mastertable,df_webtable,df_loyaltytable,prod_pred_fold)
pred_against_cids['customer_id'] = prod_pred_df['customer_id']
prod_pred_df = standardization(prod_pred_df)
prod_pred_df,OET = Onehotencoding(prod_pred_df)
prod_pred_df = prod_pred_df[prod_training_df.columns]


/var/tmp/ipykernel_2342/3307221883.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test_df = df_mastertable[df_webtable['date_of_visit'].between(fold['test_start_date'],fold['test_end_date'])]


In [61]:
final_model.fit(prod_training_df,prod_training_labels['target'])

GradientBoostingClassifier(learning_rate=0.05, loss='exponential',
                           min_samples_leaf=2, min_samples_split=4,
                           n_estimators=1000, subsample=0.75)

In [62]:
pred_against_cids['predictions'] = final_model.predict(prod_pred_df)
pred_against_cids['probablities'] = final_model.predict_proba(prod_pred_df)[:,1]

In [63]:
pred_against_cids['predictions'].value_counts()

predictions
0.0    1457
1.0      40
Name: count, dtype: int64

In [64]:
table_id = 'customer-marketing-analytics.customer_and_marketing_data_analysis_Models.subscription_predictions'
pred_against_cids['timestamp'] = pd.Timestamp.now( )
jobconfigs = bigquery.LoadJobConfig(autodetect=True,write_disposition='WRITE_APPEND')
res = bq_client.load_table_from_dataframe(pred_against_cids,table_id,job_config=jobconfigs)

In [65]:
table_id_acc = 'customer-marketing-analytics.customer_and_marketing_data_analysis_Models.subscription_model_Fold_Accuracy_Tracking'
jobconfigs = bigquery.LoadJobConfig(autodetect=True,write_disposition='WRITE_APPEND')
response = bq_client.load_table_from_dataframe(accuracy_tracking,table_id_acc,job_config=jobconfigs)